In [175]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [176]:
#Get the unique values for the schools
total_schools = len(school_data_complete["school_name"].unique())
#Get the unique values for the students
total_students = len(school_data_complete["student_name"])
#Calculate the total budget
total_budget = school_data_complete["budget"].unique().sum(axis=0)
#Calculate the math score average
math_average = school_data_complete["math_score"].mean()
#Calculate the reading score average
reading_average = school_data_complete["reading_score"].mean()

In [177]:
#Find the number of students passing math
total_passing_math = len(school_data_complete.loc[school_data_complete["math_score"] >= 70,["student_name","math_score"]])
#Calculate the percentage of students passing math
pct_passing_math = (total_passing_math/total_students) * 100

In [178]:
#Find the number of students passing reading
total_passing_reading = len(school_data_complete.loc[school_data_complete["reading_score"] >= 70,["student_name","reading_score"]])
#Calculate the percentage of students passing reading
pct_passing_reading = (total_passing_reading/total_students) * 100

In [179]:
#Calculate the overall passing percentage
overall_passing = (math_average + reading_average) / 2

## District Summary

In [180]:
#Create a summary table dataframe with the new column headers using the variables created
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                                 "Total Students": total_students,
                                 "Total Budget": total_budget,
                                 "Average Math Score": math_average,
                                 "Average Reading Score": reading_average,
                                 "% Passing Math": pct_passing_math,
                                 "% Passing Reading": pct_passing_reading,
                                 "% Overall Passing": overall_passing})

#Format the students and budget columns
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format) 
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format) 
#Print out the final summary
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


In [181]:
#Group by the school name and get academic averages
groupby_school = school_data_complete.groupby(["school_name"])
grouped_data = groupby_school.mean()

In [182]:
#Add the School Type to the grouped data
school_type = pd.DataFrame({"school_name": school_data["school_name"],
                               "School Type": school_data["type"]})
type_grouped_data = pd.merge(grouped_data, school_type, on=["school_name","school_name"])

In [183]:
#Caluclate the Per Student Budget and add it to the grouped data
student_budget = pd.DataFrame({"school_name": school_data["school_name"],
                               "Per Student Budget": school_data["budget"] / school_data["size"]})
budget_grouped_data = pd.merge(type_grouped_data, student_budget, on=["school_name","school_name"])

In [184]:
#Get the list of students passing math
passing_math_list = school_data_complete.loc[school_data_complete["math_score"] >= 70,["school_name","math_score"]]
#Group by the passing math list by the school and count the students passing math
passing_math_by_school = passing_math_list.groupby(["school_name"]).count()
#Calculate the percent passing math for each school
school_pct_passing_math = pd.DataFrame({"% Passing Math": (passing_math_by_school["math_score"] / grouped_data["size"]) * 100})

In [185]:
#Get the list of students passing reading
passing_reading_list = school_data_complete.loc[school_data_complete["reading_score"] >= 70,["school_name","reading_score"]]
#Group by the passing reading list by the school and count the students passing reading
passing_reading_by_school = passing_reading_list.groupby(["school_name"]).count()
#Calculate the percent passing reading for each school
school_pct_passing_reading =  pd.DataFrame({"% Passing Reading":(passing_reading_by_school["reading_score"] / grouped_data["size"]) * 100})


In [186]:
#Calculate the oveall passing score for each school
overall_passing_school = pd.DataFrame({"% Overall Passing Rate":(school_pct_passing_math["% Passing Math"] + school_pct_passing_reading["% Passing Reading"]) / 2})
#Combine the percentages in to one data frame
add_academic_scores = pd.merge(school_pct_passing_math, school_pct_passing_reading, on=["school_name"])
all_school_percentages = pd.merge(add_academic_scores, overall_passing_school, on=["school_name"])

In [187]:
#Add the school percentages to the grouped data
final_grouped_data = pd.merge(budget_grouped_data, all_school_percentages, on=["school_name"])

## School Summary

In [188]:
#Create a dataframe with the School Summary
school_summary = pd.DataFrame({"School Name": final_grouped_data["school_name"],
                               "School Type": final_grouped_data["School Type"],
                               "Total Students": final_grouped_data["size"],
                               "Total School Budget": final_grouped_data["budget"],
                               "Per Student Budget":final_grouped_data["Per Student Budget"],
                               "Average Math Score": final_grouped_data["math_score"],
                               "Average Reading Score": final_grouped_data["reading_score"],
                               "Per Student Budget": final_grouped_data["Per Student Budget"],
                               "% Passing Math": final_grouped_data["% Passing Math"],
                               "% Passing Reading": final_grouped_data["% Passing Reading"],
                               "% Overall Passing Rate": final_grouped_data["% Overall Passing Rate"]}).set_index("School Name", drop=True) 

#Format the student and budget columns
school_summary["Total Students"] = school_summary["Total Students"].map("{:,.0f}".format) 
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format) 
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format) 
#Remove the index column name and print out the final summary
school_summary.index.name = None
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

In [189]:
#Sort the School Summary by top performing schools 
top_performing_schools = school_summary.sort_values("% Overall Passing Rate", ascending=False)
top_performing_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [190]:
#Sort the School Summary by bottom performing schools 
bottom_performing_schools = school_summary.sort_values("% Overall Passing Rate")
bottom_performing_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [191]:
#Calculate the average math score by school for 9th grade
math_score_9th = school_data_complete.loc[school_data_complete["grade"] == "9th", ["school_name","math_score"]]
school_math_9th = math_score_9th.groupby(["school_name"])
avg_math_9th = school_math_9th.mean()


In [192]:
#Calculate the average math score by school for 10th grade
math_score_10th = school_data_complete.loc[school_data_complete["grade"] == "10th", ["school_name","math_score"]]
school_math_10th = math_score_10th.groupby(["school_name"])
avg_math_10th = school_math_10th.mean()

In [193]:
#Calculate the average math score by school for 11th grade
math_score_11th = school_data_complete.loc[school_data_complete["grade"] == "11th", ["school_name","math_score"]]
school_math_11th = math_score_11th.groupby(["school_name"])
avg_math_11th = school_math_11th.mean()

In [194]:
#Calculate the average math score by school for 12th grade
math_score_12th = school_data_complete.loc[school_data_complete["grade"] == "12th", ["school_name","math_score"]]
school_math_12th = math_score_12th.groupby(["school_name"])
avg_math_12th = school_math_12th.mean()

## Average Math Score by Grade

In [195]:
math_score_by_grade = pd.DataFrame({"9th": avg_math_9th["math_score"],
                                    "10th": avg_math_10th["math_score"],
                                    "11th": avg_math_10th["math_score"],
                                    "12th": avg_math_10th["math_score"]})
#Remove the index column name and print out the final summary
math_score_by_grade.index.name = None
math_score_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,76.996772,76.996772
Cabrera High School,83.094697,83.154506,83.154506,83.154506
Figueroa High School,76.403037,76.539974,76.539974,76.539974
Ford High School,77.361345,77.672316,77.672316,77.672316
Griffin High School,82.044010,84.229064,84.229064,84.229064
Hernandez High School,77.438495,77.337408,77.337408,77.337408
Holden High School,83.787402,83.429825,83.429825,83.429825
Huang High School,77.027251,75.908735,75.908735,75.908735
Johnson High School,77.187857,76.691117,76.691117,76.691117
Pena High School,83.625455,83.372000,83.372000,83.372000


In [196]:
#Calculate the average reading score by school for 9th grade
reading_score_9th = school_data_complete.loc[school_data_complete["grade"] == "9th", ["school_name","reading_score"]]
school_reading_9th = reading_score_9th.groupby(["school_name"])
avg_reading_9th = school_reading_9th.mean()

In [197]:
#Calculate the average reading score by school for 9th grade
reading_score_10th = school_data_complete.loc[school_data_complete["grade"] == "10th", ["school_name","reading_score"]]
school_reading_10th = reading_score_10th.groupby(["school_name"])
avg_reading_10th = school_reading_10th.mean()

In [198]:
#Calculate the average reading score by school for 9th grade
reading_score_11th = school_data_complete.loc[school_data_complete["grade"] == "11th", ["school_name","reading_score"]]
school_reading_11th = reading_score_11th.groupby(["school_name"])
avg_reading_11th = school_reading_11th.mean()

In [199]:
#Calculate the average reading score by school for 9th grade
reading_score_12th = school_data_complete.loc[school_data_complete["grade"] == "12th", ["school_name","reading_score"]]
school_reading_12th = reading_score_12th.groupby(["school_name"])
avg_reading_12th = school_reading_12th.mean()


## Average Reading Score by Grade 

In [200]:
reading_score_by_grade = pd.DataFrame({"9th": avg_reading_9th["reading_score"],
                                       "10th": avg_reading_10th["reading_score"],
                                       "11th": avg_reading_10th["reading_score"],
                                       "12th": avg_reading_10th["reading_score"]})
#Remove the index column name and print out the final summary
reading_score_by_grade.index.name = None
reading_score_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.907183,80.907183
Cabrera High School,83.676136,84.253219,84.253219,84.253219
Figueroa High School,81.198598,81.408912,81.408912,81.408912
Ford High School,80.632653,81.262712,81.262712,81.262712
Griffin High School,83.369193,83.706897,83.706897,83.706897
Hernandez High School,80.866860,80.660147,80.660147,80.660147
Holden High School,83.677165,83.324561,83.324561,83.324561
Huang High School,81.290284,81.512386,81.512386,81.512386
Johnson High School,81.260714,80.773431,80.773431,80.773431
Pena High School,83.807273,83.612000,83.612000,83.612000


## Scores by School Spending

In [201]:
# Creating the bins and groups
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [202]:
#Using the grouped data from the School Summary
final_grouped_data["Spending Ranges (Per Student)"] = pd.cut(final_grouped_data["Per Student Budget"], spending_bins, labels=group_names)
groupby_spending = final_grouped_data.groupby("Spending Ranges (Per Student)")
school_spending_groups = groupby_spending.mean()
school_spending_groups
#Selecting only the columns needed for analysis
scores_by_school_spending = pd.DataFrame({"Average Math Score": school_spending_groups["math_score"],
                                         "Average Reading Score": school_spending_groups["reading_score"],
                                         "% Passing Math": school_spending_groups["% Passing Math"],
                                         "% Passing Reading": school_spending_groups["% Passing Reading"],
                                         "% Overall Passing Rate": school_spending_groups["% Overall Passing Rate"]})
scores_by_school_spending


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

In [203]:
# Creating the bins and group names
size_bins = [0, 1000, 2000, 5000]
group_names = ["4A (<1000)", "5A (1000-2000)", "6A (2000-5000)"]

In [204]:
#Using the grouped data from the School Summary
final_grouped_data["School Size"] = pd.cut(final_grouped_data["size"], size_bins, labels=group_names)
groupby_size = final_grouped_data.groupby("School Size")
school_size_groups = groupby_size.mean()
school_size_groups
#Selecting only the columns needed for analysis
scores_by_school_size = pd.DataFrame({"Average Math Score": school_size_groups["math_score"],
                                         "Average Reading Score": school_size_groups["reading_score"],
                                         "% Passing Math": school_size_groups["% Passing Math"],
                                         "% Passing Reading": school_size_groups["% Passing Reading"],
                                         "% Overall Passing Rate": school_size_groups["% Overall Passing Rate"]})
scores_by_school_size


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
4A (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
5A (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
6A (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [205]:
#Using the grouped data from the School Summary
groupby_type = final_grouped_data.groupby("School Type")
school_type_groups = groupby_type.mean()
school_type_groups
#Selecting only the columns needed for analysis
scores_by_school_type = pd.DataFrame({"Average Math Score": school_type_groups["math_score"],
                                         "Average Reading Score": school_type_groups["reading_score"],
                                         "% Passing Math": school_type_groups["% Passing Math"],
                                         "% Passing Reading": school_type_groups["% Passing Reading"],
                                         "% Overall Passing Rate": school_type_groups["% Overall Passing Rate"]})
scores_by_school_type


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
